# COURSE:   PGP [AI&ML]

## Learner :  Chaitanya Kumar Battula
## Module  : RNN
## Topic   : Implement Deep Q-Learning method in a CartPole-V0 environment with Keras.

Apply the following hyperparameters:

Memory = 2000 </br>
Gamma = 0.95  </br> 
Maximum epsilon = 1.0  </br>
Minimum epsilon_min = 0.01 </br>
Decaying epsilon = 0.99 </br>
Learning rate = 0.001 </br>

### Import the libraries

In [7]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K

import tensorflow as tf

In [8]:
env = gym.make('CartPole-v1')

In [9]:
state_size = env.observation_space.shape[0]

In [10]:
state_size

4

### Define number of episodes

In [17]:
EPISODES = 2

### Create a DQN agent

In [27]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    
        self.epsilon = 1.0  
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _huber_loss(self, y_true, y_pred, clip_delta=1.0):
        error = y_true - y_pred
        cond  = K.abs(error) <= clip_delta

        squared_loss = 0.5 * K.square(error)
        quadratic_loss = 0.5 * K.square(clip_delta) + clip_delta * (K.abs(error) - clip_delta)

        return K.mean(tf.where(cond, squared_loss, quadratic_loss))

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss=self._huber_loss,
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0]) 

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        print("minibatch")
        print(minibatch)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            print("target")
            print(target)
            if done:
                target[0][action] = reward
            else:
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    done = False
    batch_size = 32
   
    for e in range(EPISODES):
        print("Episode No. " + str(e))
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            print("STEP No : " + str(time))
            if done:
                agent.update_target_model()
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break
            if len(agent.memory) > batch_size:
                print("replaying")
                agent.replay(batch_size)
        

Episode No. 0
STEP No : 0
STEP No : 1
STEP No : 2
STEP No : 3
STEP No : 4
STEP No : 5
STEP No : 6
STEP No : 7
STEP No : 8
STEP No : 9
STEP No : 10
STEP No : 11
STEP No : 12
STEP No : 13
STEP No : 14
STEP No : 15
STEP No : 16
STEP No : 17
STEP No : 18
STEP No : 19
STEP No : 20
STEP No : 21
STEP No : 22
STEP No : 23
STEP No : 24
STEP No : 25
STEP No : 26
STEP No : 27
episode: 0/2, score: 27, e: 1.0
Episode No. 1
STEP No : 0
STEP No : 1
STEP No : 2
STEP No : 3
STEP No : 4
replaying
minibatch
[(array([[-0.00055433, -0.00204169,  0.00381884,  0.11003557]]), 0, 1.0, array([[-0.00059516, -0.19721815,  0.00601955,  0.40392087]]), False), (array([[-0.0392367 ,  0.00076665,  0.03923149,  0.04812754]]), 1, 1.0, array([[-0.03922137,  0.19530474,  0.04019404, -0.23192396]]), False), (array([[-0.0004236 ,  0.19288332, -0.00296518, -0.17830203]]), 0, 1.0, array([[ 0.00343407, -0.00219607, -0.00653122,  0.11344401]]), False), (array([[-0.00037732, -0.00231397, -0.00528606,  0.11604391]]), 1, 1.0, arra

target
[[0.17702734 0.04089829]]
target
[[0.2749429  0.05088406]]
target
[[0.13258693 0.04404882]]
target
[[0.21827355 0.05085549]]
target
[[0.30730003 0.05928314]]
target
[[0.28495297 0.06677418]]
target
[[0.10720319 0.08866072]]
target
[[0.12567109 0.08087647]]
target
[[0.1466984  0.05946194]]
target
[[0.1811645  0.05721364]]
target
[[0.1250025  0.07504641]]
target
[[0.15640633 0.06644105]]
target
[[0.23470393 0.08678658]]
target
[[0.15330526 0.09329123]]
target
[[0.24652496 0.09234563]]
target
[[0.32769504 0.08110547]]
target
[[0.15031826 0.08596671]]
target
[[0.4227109  0.09471786]]
target
[[0.15597276 0.1169526 ]]
target
[[0.3753082  0.11440566]]
target
[[0.43123296 0.11020466]]
target
[[0.505571   0.13064149]]
target
[[0.16674347 0.09824154]]
target
[[0.49090335 0.14885247]]
STEP No : 6
replaying
minibatch
[(array([[-0.00037732, -0.00231397, -0.00528606,  0.11604391]]), 1, 1.0, array([[-0.0004236 ,  0.19288332, -0.00296518, -0.17830203]]), False), (array([[-0.00055433, -0.0020416

target
[[0.6534003 0.519241 ]]
target
[[0.6687423  0.53347135]]
target
[[0.5579246 0.4751032]]
target
[[0.6014511  0.36591536]]
target
[[0.5237262  0.34669125]]
target
[[0.60487777 0.5136764 ]]
target
[[0.913862  0.7035372]]
target
[[0.9554841 0.7350553]]
target
[[0.47015733 0.4371052 ]]
target
[[0.47189844 0.4475602 ]]
target
[[0.5915182  0.41010046]]
target
[[0.53181386 0.49408117]]
target
[[0.5439411  0.46422935]]
target
[[0.5015302 0.3996547]]
target
[[0.9464295 0.8018739]]
target
[[0.52908134 0.5183836 ]]
target
[[0.7370474 0.6793228]]
target
[[0.7922508  0.57168186]]
target
[[0.76035976 0.7004535 ]]
target
[[0.5701026  0.47999948]]
target
[[0.58260965 0.49070826]]
target
[[0.5867416 0.5779244]]
target
[[1.0125226 0.7250333]]
STEP No : 8
replaying
minibatch
[(array([[-0.00453952, -0.39242496,  0.01409797,  0.69849551]]), 1, 1.0, array([[-0.01238802, -0.19750129,  0.02806788,  0.41028373]]), False), (array([[-0.01199321,  0.38801198,  0.00765896, -0.47116894]]), 0, 1.0, array([[-0.

target
[[0.9669344 0.9847772]]
target
[[1.136332  1.1252832]]
STEP No : 10
replaying
minibatch
[(array([[-0.03532263,  0.19422685,  0.0370188 , -0.20809469]]), 1, 1.0, array([[-0.03143809,  0.38880044,  0.03285691, -0.48887408]]), False), (array([[-0.08784043,  0.17826565,  0.18184524,  0.14635309]]), 0, 1.0, array([[-0.08427512, -0.01893144,  0.1847723 ,  0.49043988]]), False), (array([[-0.06358582, -0.59274946,  0.122439  ,  1.11286834]]), 1, 1.0, array([[-0.07544081, -0.39942939,  0.14469637,  0.86096631]]), False), (array([[ 0.02289797,  0.19054869, -0.08321965, -0.31435637]]), 0, 1.0, array([[ 0.02670895, -0.00329529, -0.08950678, -0.04903502]]), False), (array([[-0.02366209,  0.5834438 ,  0.02307942, -0.77102335]]), 0, 1.0, array([[-0.01199321,  0.38801198,  0.00765896, -0.47116894]]), False), (array([[-0.01633805, -0.39300969,  0.03627356,  0.71168169]]), 0, 1.0, array([[-0.02419824, -0.58861465,  0.05050719,  1.01555822]]), False), (array([[-0.03483562,  0.18804353,  0.00150849

target
[[0.8769069  0.93505245]]
target
[[0.9973847 1.0790198]]
target
[[0.91905594 0.9398253 ]]
target
[[1.0967672 1.1353871]]
target
[[1.1143368 1.1909325]]
target
[[0.94825494 1.0098746 ]]
target
[[1.1244339 1.1749289]]
target
[[1.2584339 1.295937 ]]
target
[[1.1741663 1.1516632]]
target
[[1.022945  1.0251298]]
target
[[0.9449018 1.0123746]]
target
[[0.8841052  0.94347286]]
target
[[1.3199956 1.2646487]]
target
[[1.0565226 1.1000006]]
target
[[0.9639237 0.9657674]]
target
[[0.9075899 0.9641625]]
target
[[0.90142107 0.9297692 ]]
target
[[0.8966452  0.96378666]]
target
[[0.92810625 0.95289844]]
target
[[1.120121  1.1036783]]
target
[[0.97979516 0.9866699 ]]
target
[[0.92245984 0.9448686 ]]
target
[[0.9854607 0.9907244]]
target
[[0.89405906 0.93864304]]
STEP No : 12
replaying
minibatch
[(array([[-0.02419824, -0.58861465,  0.05050719,  1.01555822]]), 1, 1.0, array([[-0.03597054, -0.39420128,  0.07081836,  0.73915284]]), False), (array([[-0.05565908, -0.39633683,  0.10666653,  0.78862333

...

target
[[1.0570519 0.9741024]]
target
[[1.0355494 0.9949682]]
target
[[1.0194958 1.0920448]]
target
[[0.98381335 0.9993342 ]]
target
[[1.0508783 0.9718991]]
target
[[1.0529157 1.0088793]]
target
[[1.0803388 0.9811251]]
target
[[1.0777342 1.0233762]]
target
[[1.131438  0.9899311]]
target
[[1.0411702 1.0535804]]
target
[[1.0434088  0.97396845]]
target
[[1.1816304 1.0307106]]
target
[[1.1135752 1.0489709]]
target
[[1.008188  1.0056505]]
target
[[1.0832063 1.0321027]]
target
[[0.98797846 0.97837317]]
target
[[1.3400997 1.1508713]]
target
[[1.092016  1.0470785]]
target
[[0.9132639 1.0837617]]
target
[[1.1220024 1.0431923]]
target
[[0.9585339 1.0095237]]
target
[[0.8766654  0.99621713]]
target
[[1.0850728  0.99617285]]
target
[[0.96506375 0.9633247 ]]
target
[[1.0145594 1.0005145]]
target
[[0.91428524 1.0470648 ]]
target
[[0.923375  1.0042146]]
target
[[1.0048441 0.9935092]]
target
[[1.004857  1.1354175]]
target
[[0.9311514 1.0271572]]
target
[[0.94157475 0.9110032 ]]
STEP No : 31
replaying


target
[[1.0637555 1.0472126]]
target
[[1.1053587 0.9945968]]
target
[[0.99805003 1.0135882 ]]
target
[[1.0355582  0.95847595]]
target
[[1.1710594 1.0235875]]
target
[[1.0082685 1.0351703]]
target
[[1.0466859  0.95959604]]
target
[[1.0867641 0.9807006]]
target
[[1.0635929  0.96922374]]
target
[[1.1079314 1.0440131]]
target
[[0.99167687 1.0328839 ]]
target
[[1.0833479 1.0288504]]
target
[[1.1843561 1.141752 ]]
target
[[1.0097433 1.0823534]]
target
[[1.1246338 1.0523348]]
target
[[1.0338933 1.0115358]]
target
[[1.1156107 1.0261858]]
target
[[1.1420536 1.1487772]]
target
[[1.0562025 1.0194694]]
target
[[0.9974364 1.0587112]]
target
[[0.972044   0.97176534]]
target
[[1.0980589 1.039841 ]]
target
[[0.99828935 1.0062973 ]]
target
[[1.0043046 1.0001192]]
target
[[1.0814096 0.983626 ]]
target
[[1.0359108  0.97539485]]
target
[[1.0528796 0.9831363]]
target
[[1.033716  1.0449891]]
STEP No : 34
replaying
minibatch
[(array([[ 0.02664304,  0.19298851, -0.09048748, -0.36856199]]), 0, 1.0, array([[ 0

target
[[1.0264544 1.0119171]]
target
[[1.0478637 1.0320247]]
target
[[1.1245799 1.0111947]]
target
[[1.1460361 1.0524944]]
target
[[1.0302134 1.0342832]]
target
[[1.0136619 0.9996672]]
target
[[1.0338755 1.0254894]]
target
[[1.0539914 0.9899585]]
target
[[1.0170308 0.9483944]]
target
[[1.2193342 1.0708165]]
target
[[1.1398698 1.0207682]]
target
[[1.0925735 1.1580483]]
target
[[1.0169386 1.0204855]]
target
[[1.0683756 1.1762123]]
target
[[1.0264386 0.9435328]]
target
[[1.0974753 1.0482614]]
target
[[0.9676267 1.018148 ]]
target
[[1.0379355  0.95396894]]
target
[[1.0117579 1.0141231]]
target
[[0.9832783 1.0439341]]
target
[[1.0459988 1.0233177]]
target
[[1.0200248 0.946259 ]]
STEP No : 36
replaying
minibatch
[(array([[ 0.00774228,  0.00989467, -0.09658955, -0.34128887]]), 0, 1.0, array([[ 0.00794018, -0.18372984, -0.10341532, -0.08055907]]), False), (array([[-0.03469429, -0.00706662,  0.00082213,  0.03431805]]), 1, 1.0, array([[-0.03483562,  0.18804353,  0.00150849, -0.25810537]]), Fals

target
[[1.3473753 1.1222904]]
target
[[0.983426  0.9661611]]
target
[[1.0011189 1.0560788]]
target
[[0.9960986 0.9824541]]
target
[[1.119371 1.009006]]
target
[[1.0114105 0.9908742]]
target
[[1.1942563 1.1180583]]
target
[[1.0576841 1.0103742]]
target
[[1.1174043 1.0249549]]
target
[[1.0053033 1.0647479]]
target
[[1.1201571 0.9638154]]
target
[[1.1183212 1.0186874]]
target
[[1.1827785 1.0112798]]
target
[[1.0442137 1.0199019]]
target
[[1.1289369 1.0278329]]
target
[[1.0015911  0.99917614]]
target
[[1.0204371 1.0293807]]
target
[[0.91431344 1.0748917 ]]
target
[[1.0525539 1.0220959]]
target
[[0.9955581  0.96494657]]
target
[[1.1513337 1.0124568]]
target
[[1.0287619 1.0277448]]
target
[[0.91612077 0.9010071 ]]
target
[[0.9187833 1.0337207]]
target
[[0.99569124 1.0116321 ]]
target
[[0.9608314  0.99083525]]
target
[[0.93045175 0.96857154]]
target
[[0.9889882 0.947625 ]]
target
[[0.9594392 0.9885976]]
target
[[0.982608  1.0034615]]
target
[[0.95302385 1.018633  ]]
STEP No : 39
replaying
mi

target
[[1.1869181 1.0599356]]
target
[[1.3671505 1.204082 ]]
target
[[1.0119146  0.98967975]]
target
[[1.1035798 1.0449722]]
target
[[1.0083101 1.0754765]]
target
[[1.1228439 1.0306242]]
target
[[1.0872945  0.99033016]]
target
[[1.044415  1.1183465]]
target
[[1.0836505 1.0073916]]
target
[[1.090403  1.0368888]]
target
[[1.1214776 1.0543195]]
target
[[1.1251223 1.1636226]]
target
[[0.9768014 1.0031362]]
target
[[1.0292705 1.0525032]]
target
[[1.097812  0.9830358]]
target
[[1.108442  1.0304159]]
target
[[1.1756209 1.1424489]]
target
[[0.9945061  0.93796587]]
target
[[1.092754   0.98730135]]
target
[[1.058074  1.0074008]]
target
[[0.9940982 0.9711144]]
target
[[1.0086815 0.9934724]]
target
[[1.0391963 1.0077912]]
target
[[1.124863  1.0151489]]
target
[[1.0072105 1.0645564]]
STEP No : 41
replaying
minibatch
[(array([[ 0.00794018, -0.18372984, -0.10341532, -0.08055907]]), 0, 1.0, array([[ 0.00426558, -0.37722899, -0.10502651,  0.17778903]]), False), (array([[-0.03469429, -0.00706662,  0.00